In [23]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE

In [4]:
bank = pd.read_csv(r"D:\Analytix Lab AI Course Files\Python for Data Science Module 1\ML Ops\Banking Application\bankloans.csv")

In [5]:
bank

,age,ed,employ,address,income,debtinc,creddebt,othdebt,default
0,41,3,17,12,176,9.3,11.359392,5.008608,1.0
1,27,1,10,6,31,17.3,1.362202,4.000798,0.0
2,40,1,15,14,55,5.5,0.856075,2.168925,0.0
3,41,1,15,14,120,2.9,2.658720,0.821280,0.0
4,24,2,2,0,28,17.3,1.787436,3.056564,1.0
...,...,...,...,...,...,...,...,...,...
845,34,1,12,15,32,2.7,0.239328,0.624672,NaN
846,32,2,12,11,116,5.7,4.026708,2.585292,NaN
847,48,1,13,11,38,10.8,0.722304,3.381696,NaN
848,35,2,1,11,24,7.8,0.417456,1.454544,NaN


In [6]:
bank.isna().sum()

age           0
ed            0
employ        0
address       0
income        0
debtinc       0
creddebt      0
othdebt       0
default     150
dtype: int64

In [7]:
bank.dropna(inplace=True)

In [9]:
bank.info()

<class 'pandas.core.frame.DataFrame'>
Index: 700 entries, 0 to 699
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       700 non-null    int64  
 1   ed        700 non-null    int64  
 2   employ    700 non-null    int64  
 3   address   700 non-null    int64  
 4   income    700 non-null    int64  
 5   debtinc   700 non-null    float64
 6   creddebt  700 non-null    float64
 7   othdebt   700 non-null    float64
 8   default   700 non-null    float64
dtypes: float64(4), int64(5)
memory usage: 54.7 KB


In [11]:
## Outlier Treatment

def outlier_cap(x):
    x = x.clip(upper=x.quantile(0.99), lower=x.quantile(0.01))
    return x

bank = bank.apply(lambda x: outlier_cap(x))

In [13]:
## Separating Target and Independent Variables
y = bank['default'].values
x = bank.drop(columns=['default'])

In [16]:
## Train Test Split

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.33, random_state=42)

In [17]:
rfe = RFE(estimator=RandomForestClassifier(), n_features_to_select=5)

In [18]:
rfe_model = rfe.fit(X_train, y_train)

In [19]:
X_train.columns[rfe_model.get_support()]

Index(['age', 'employ', 'debtinc', 'creddebt', 'othdebt'], dtype='object')

In [20]:
from sklearn.model_selection import GridSearchCV

In [25]:


params = {
    "n_estimators": [10,20,30],
    "max_depth": [1,2,3]
}

rf_estimator = RandomForestClassifier(n_jobs=-1, oob_score=True, random_state=1024)

In [26]:
rf_cv = GridSearchCV(rf_estimator, params, cv=5)
rf_cv.fit(X_train, y_train)

C:\Users\samme\anaconda3\Lib\site-packages\sklearn\ensemble\_forest.py:583: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(
C:\Users\samme\anaconda3\Lib\site-packages\sklearn\ensemble\_forest.py:583: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(
C:\Users\samme\anaconda3\Lib\site-packages\sklearn\ensemble\_forest.py:583: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(
C:\Users\samme\anaconda3\Lib\site-packages\sklearn\ensemble\_forest.py:583: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(
C:\Users\samme\anaconda3\Lib\site-packages\sklearn\ensemble\_forest.py:583: UserWarning: Some inputs do not have OOB scores. This probab

GridSearchCV(cv=5,
             estimator=RandomForestClassifier(n_jobs=-1, oob_score=True,
                                              random_state=1024),
             param_grid={'max_depth': [1, 2, 3], 'n_estimators': [10, 20, 30]})

In [27]:
rf_cv.best_estimator_

RandomForestClassifier(max_depth=3, n_estimators=10, n_jobs=-1, oob_score=True,
                       random_state=1024)

In [29]:
from sklearn.metrics import classification_report

In [31]:
print(classification_report(y_train, rf_cv.best_estimator_.predict(X_train)))

              precision    recall  f1-score   support

         0.0       0.81      0.96      0.88       344
         1.0       0.76      0.38      0.50       125

    accuracy                           0.80       469
   macro avg       0.78      0.67      0.69       469
weighted avg       0.79      0.80      0.78       469



In [32]:
print(classification_report(y_test, rf_cv.best_estimator_.predict(X_test)))

              precision    recall  f1-score   support

         0.0       0.79      0.95      0.86       173
         1.0       0.62      0.22      0.33        58

    accuracy                           0.77       231
   macro avg       0.70      0.59      0.60       231
weighted avg       0.74      0.77      0.73       231



In [33]:
import pickle

In [35]:
filename = "RFClassifier.pkl"
pickle.dump(rf_cv.best_estimator_, open(filename, "wb"))

In [37]:
retrieved_model = pickle.load(open(filename, 'rb'))

In [53]:
rf_cv.best_estimator_.feature_names_in_

array(['age', 'ed', 'employ', 'address', 'income', 'debtinc', 'creddebt',
       'othdebt'], dtype=object)

8